In [1]:
import polars as pl
import pandas as pd
import numpy as np

In [2]:
df11 = pl.scan_csv('38430-0001-Data.tsv',separator='\t',dtypes={"IDHH": pl.Categorical,"IDPER": pl.Categorical},ignore_errors = True)
result_df = df11.collect()
df_1 = result_df

In [3]:
len(df_1)

3932046

In [4]:
df_1['YEAR'].dtype

Int64

In [5]:
final_df=df_1.filter(df_1['YEAR'].is_in([2019,2021]))

In [6]:
final_df

YEAR,YEARQ,YEARQ_ORIG,YEAR_ORIG,IDHH,V2001,V2002,V2003,V2004,V2005,V2006,V2007,V2008,V2009,V2010,V2011,V2012,V2013,V2014,V2015,V2016,V2017,V2143,V2018,V2019,V2020,V2021,V2022,V2023,V2024,V2025,V2025A,V2025B,V2026,SC214A,V2027,V2028,…,V2124,V2125,V2126A,V2126B,V2127A,V2127B,V2128A,V2128B,V2129,V2130,V2131,V2132,V2133,V2134,V2135,V2136,V2137,V2138,V2139,V2140,V2141,V2142,FRCODE,COHORT,TIS,ADDR_TIS,HH_TIS,HHINTVNUM,TYPEINTV_TIS1,TYPEINTV_TIS2,TYPEINTV_TIS3,TYPEINTV_TIS4,TYPEINTV_TIS5,TYPEINTV_TIS6,TYPEINTV_TIS7,HHBOUNDED,VFLAG
i64,f64,i64,i64,cat,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,…,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
2019,2019.1,9999,9999,"""16020002581336…",2,2,191,16,2000258133680568236,1,-2,17,0,9,9,99,999,9,9,1,1,-1,9,7,1,1,9,9,99,9,8,8,98,12,1,24,…,2,1,-2,19,-2,1,-2,1,1,1,2019,9,9,98,98,98,98,98,98,98,98,2019,-1,-1,5,5,5,0,2,2,2,2,3,9,9,0,9
2019,2019.1,9999,9999,"""16020002581636…",2,2,191,16,2000258163682568236,1,-2,11,0,9,9,99,999,2,9,1,1,-1,9,7,1,1,9,9,99,9,1,1,98,13,1,30,…,2,1,-2,16,-2,1,-2,1,2,1,2019,9,9,98,98,98,98,98,98,98,98,2019,-1,-1,4,4,4,1,2,1,2,2,9,9,9,0,9
2019,2019.1,9999,9999,"""16020002583594…",2,2,191,16,2000258359426568236,1,-2,17,0,9,9,99,999,1,9,1,1,-1,9,7,1,1,9,9,99,9,2,2,13,13,1,30,…,2,1,-2,16,-2,1,-2,1,2,1,2019,9,9,98,98,98,98,98,98,98,98,2019,-1,-1,5,5,5,2,1,2,2,1,2,9,9,1,9
2019,2019.1,9999,9999,"""16020002586836…",2,2,191,16,2000258683621568236,1,-2,11,0,1,1,1,998,1,1,1,1,-1,9,7,1,1,1,1,1,9,2,2,98,11,99,99,…,2,1,-2,0,-2,1,-2,1,2,1,2019,4,4,98,98,98,98,98,98,98,98,2019,-1,-1,4,4,4,4,1,1,1,1,9,9,9,1,1
2019,2019.1,9999,9999,"""16020002651346…",2,2,191,16,2000265134680568236,1,-2,17,0,1,1,1,998,1,1,1,1,-1,9,7,1,1,8,8,1,9,2,2,17,17,99,99,…,2,1,-2,16,-2,1,-2,1,2,1,2019,4,4,98,98,98,98,98,98,98,98,2019,-1,-1,5,5,5,3,2,2,1,1,1,9,9,1,1
2019,2019.1,9999,9999,"""16020002655246…",2,2,191,16,2000265524673568236,1,-2,11,0,1,3,1,998,2,2,1,1,-1,9,7,1,1,8,8,6,1,2,2,15,15,99,99,…,2,1,-2,0,-2,1,-2,1,2,1,2019,4,4,98,98,98,98,98,98,98,98,2019,-1,-1,4,4,4,1,3,2,2,1,9,9,9,0,1
2019,2019.1,9999,9999,"""16020002841646…",2,1,191,16,2000284164631568236,1,-2,17,0,9,9,99,999,9,9,9,1,-1,9,7,1,1,9,9,99,9,8,8,98,13,1,29,…,2,1,-2,13,-2,1,-2,1,2,1,2019,9,9,98,98,98,98,98,98,98,98,2019,-1,-1,5,5,5,0,2,2,2,2,2,9,9,0,9
2019,2019.1,9999,9999,"""16020002846846…",2,25,191,16,2000284684650568236,1,-2,31,0,1,1,2,998,1,1,1,1,-1,9,7,1,1,1,1,1,9,2,2,15,15,99,99,…,2,1,-2,13,-2,1,-2,1,2,3,2019,4,4,98,98,98,98,98,98,98,98,2019,-1,-1,4,4,4,4,1,1,1,1,9,9,9,1,1
2019,2019.1,9999,9999,"""16020013580265…",2,9,191,16,2001358026592568234,1,-2,37,0,9,9,99,999,9,9,1,1,-1,9,7,1,1,9,9,99,9,8,8,98,16,3,9,…,2,1,-2,16,-2,1,-2,1,2,3,2019,9,9,98,98,98,98,98,98,98,98,2019,-1,-1,5,5,5,0,2,2,2,2,2,9,9,0,9


In [7]:
#Converting to pandas as it is a smaller dataframe
final_df = final_df.to_pandas()

## Removing Columns

In [8]:
#Removing columns with a single entry
for a in final_df.columns:
    unique_vals = final_df[a].unique()

    if len(unique_vals) == 1:
        final_df = final_df.drop(a,axis=1)

print(final_df)

        YEAR   YEARQ                       IDHH  V2002  V2003  V2004  \
0       2019  2019.1  1602000258133680568236117      2    191     16   
1       2019  2019.1  1602000258163682568236111      2    191     16   
2       2019  2019.1  1602000258359426568236117      2    191     16   
3       2019  2019.1  1602000258683621568236111      2    191     16   
4       2019  2019.1  1602000265134680568236117      2    191     16   
...      ...     ...                        ...    ...    ...    ...   
510389  2021  2021.4  2095986865027937564558151  64192    214     20   
510390  2021  2021.4  2095986865192721564558142  64193    214     20   
510391  2021  2021.4  2095986865412780564558142  64194    214     20   
510392  2021  2021.4  2095986865413466564558161  64195    214     20   
510393  2021  2021.4  2095986865415313564558141  64196    214     20   

               V2005  V2006  V2008  V2010  ...  HHINTVNUM  TYPEINTV_TIS1  \
0       2.000258e+18      1     17      9  ...          0  

In [9]:
final_df.columns

Index(['YEAR', 'YEARQ', 'IDHH', 'V2002', 'V2003', 'V2004', 'V2005', 'V2006',
       'V2008', 'V2010',
       ...
       'HHINTVNUM', 'TYPEINTV_TIS1', 'TYPEINTV_TIS2', 'TYPEINTV_TIS3',
       'TYPEINTV_TIS4', 'TYPEINTV_TIS5', 'TYPEINTV_TIS6', 'TYPEINTV_TIS7',
       'HHBOUNDED', 'VFLAG'],
      dtype='object', length=130)

#Columns to delete based on context
V2002 - Can get this info from IDHH
V2003 - Can get this from YEARQ
V2004, V2005, V2008, V2010, V2011, V2012, V2013 - Related to survey
V2014, V2016, V2018, V2020, V2026 - Using the most updated columns
V2143 - Can get this info from V2017

In [10]:
#Recoding Household number as Original - 1 or Replacement - 2
final_df.loc[final_df['V2006']>=2,'V2006']=2
final_df['V2006'].value_counts()

V2006
1    455136
2     55258
Name: count, dtype: int64

In [11]:
#Recoding Tenure as Owned - 1 or Rented - 2
final_df.loc[(final_df['V2015']==2) | (final_df['V2015']==3),'V2015']=2
final_df['V2015'].value_counts()

V2015
1    206075
9    205180
2     99139
Name: count, dtype: int64

In [14]:
#Combining blanks and residue as one entry in Farm Sales
final_df.loc[(final_df['V2019']==7) | (final_df['V2019']==8),'V2019']=8
final_df['V2019'].value_counts()

V2019
8    395469
2    104645
1      5407
9      4873
Name: count, dtype: int64

In [15]:
final_df['V2021'].value_counts()

V2021
1     482453
5      11428
11      4923
99      4873
12      2315
6       1763
10      1077
7        526
2        324
4        267
8        250
3        101
9         94
Name: count, dtype: int64

In [16]:
#Combining several types of housing
#hotel(2) - 2,3,9 
#mobile home(3) - 5,6,10
#quarters/dorms(4) - 4,8,11
#residue(98) - 7,12,98
final_df.loc[(final_df['V2021']==2) | (final_df['V2021']==3) | (final_df['V2021']==9),'V2021']=2
final_df.loc[(final_df['V2021']==5) | (final_df['V2021']==6) | (final_df['V2021']==10),'V2021']=3
final_df.loc[(final_df['V2021']==4) | (final_df['V2021']==8) | (final_df['V2021']==11),'V2021']=4
final_df.loc[(final_df['V2021']==7) | (final_df['V2021']==12) | (final_df['V2021']==98),'V2021']=98
final_df['V2021'].value_counts()

V2021
1     482453
3      14268
4       5440
99      4873
98      2841
2        519
Name: count, dtype: int64